# 1. 라이브러리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import math

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 불러오기

In [2]:
df = pd.read_csv('./data/visualization_train.csv')
dmg_data_path = os.listdir('./data/damage')

df : 지역별 데이터를 모두 통합한 데이터

dmg_data_path : 피해규모 데이터 경로

# 3. 전처리 함수

In [3]:
# 특별시, 광역시 데이터 추출
def metro_city(df, city, city_list):
    idx_city = list(df[df['지점명'] == city].index)
    city_list += idx_city

피해규모 데이터는 서울과 6개의 광역시에 대한 데이터입니다.

따라서 df로부터 해당 지역 데이터의 인덱스를 리스트로 저장합니다.

In [4]:
# 원하는 연도 추출
def metro_year(df, year, year_list):
    idx_year = list(df[df['연도'] == year].index)
    year_list += idx_year

피해규모 데이터는 2015, 2017, 2018, 2019, 2020년의 데이터입니다.

따라서 df로부터 해당 연도 데이터의 인덱스를 리스트로 저장합니다.

In [5]:
# 원하는 자연재난 추출
def weather_df(df, natural_disaster):
    idx = []
    for i in range(len(df)):
        if df[natural_disaster][i] == 1:
            idx.append(i)

    df = df.iloc[idx, :]
    df.reset_index(drop=True, inplace=True)
    return df

폭염, 한파, 호우, 대설 중 특정 자연재난 데이터를 추출합니다.

In [6]:
def mpc_year(df):
    mpc_list = ['서울', '인천', '대전', '대구', '부산', '울산', '광주']
    year_list = [2015, 2017, 2018, 2019, 2020]
    idx_metro = []
    idx_year = []

    # 서울 및 광역시 데이터
    for city in mpc_list:
        metro_city(df, city, idx_metro)

    df_mpc = df.iloc[idx_metro, :]
    df_mpc.reset_index(drop=True, inplace=True)

    # 연도 데이터
    for year in year_list:
        metro_year(df_mpc, year, idx_year)

    df_mpc = df_mpc.iloc[idx_year, :]
    df_mpc.reset_index(drop=True, inplace=True)

    df_mpc['원인'] = '없음'
    df_mpc['호우_인명피해'] = 0
    df_mpc['대설_인명피해'] = 0
    df_mpc['총_인명피해'] = 0
    df_mpc['호우_피해액'] = 0    
    df_mpc['대설_피해액'] = 0    
    df_mpc['총_피해액'] = 0

    return df_mpc

앞서 정의한 함수들을 이용하여 새로운 데이터프레임을 생성합니다.

지역은 서울, 인천, 대전, 대구, 부산, 울산, 광주입니다.

연도는 2015년, 2017년, 2018년, 2019년, 2020년입니다.

새로운 데이터프레임을 생성한 후, 원인과 인명피해, 피해액 컬럼을 생성합니다.

In [7]:
def nd_dmg(dmg_data_path, df, natural_disaster, standard):
    df_damage = pd.read_csv('./data/damage/' + dmg_data_path)
    df_damage = df_damage.iloc[1:, :].copy()
    df_damage.reset_index(drop=True, inplace=True)
    
    df_damage['시작_일자'] = pd.to_datetime(df_damage['시작_일자'])
    df_damage['종료_일자'] = pd.to_datetime(df_damage['종료_일자'])

    df_damage['시작_일자'] = df_damage['시작_일자'].astype(str)
    df_damage['종료_일자'] = df_damage['종료_일자'].astype(str)
    
    # 지역 컬럼 수정
    for i in range(len(df_damage)):
        df_damage['지역'][i] = df_damage['지역'][i][:2]

    # metro_city 함수 사용을 위해 컬럼명 통일
    df_damage.rename(columns={'지역' : '지점명'}, inplace=True)

    # 이재민수 type 변경
    df_damage['이재민수'] = df_damage['이재민수'].astype(int)
    
    mpc_list = ['서울', '인천', '대전', '대구', '부산', '울산', '광주']
    idx_metro = []

    for city in mpc_list:
        metro_city(df_damage, city, idx_metro)

    df_damage = df_damage.iloc[idx_metro, :]
    df_damage.reset_index(drop=True, inplace=True)

    # 인명피해 및 피해액 데이터 입력
    for i in range(len(df_damage)):
        if natural_disaster in df_damage['원인'][i]:
            df_temp = df[df['시간'].between(df_damage['시작_일자'][i], df_damage['종료_일자'][i])]
            region_list = df_temp[df_temp['지점명'] == df_damage['지점명'][i]].index
            df_region = df.iloc[region_list, :]
            df_region.reset_index(drop=True, inplace=True)
            
            for j in range(len(df_region)):
                if sum(df_region[standard]) != 0:
                    df_region[f'{natural_disaster}_인명피해'][j] = df_region[standard][j]/sum(df_region[standard]) * df_damage['이재민수'][i]
                    df_region[f'{natural_disaster}_피해액'][j] = df_region[standard][j]/sum(df_region[standard]) * df_damage['피해액'][i]

                    df[f'{natural_disaster}_인명피해'][region_list[j]] += df_region[f'{natural_disaster}_인명피해'][j]
                    df[f'{natural_disaster}_피해액'][region_list[j]] += df_region[f'{natural_disaster}_피해액'][j]
                    
                    df['원인'][region_list[j]] = natural_disaster
                else:
                    df_region[f'{natural_disaster}_인명피해'][j] = 0
                    df_region[f'{natural_disaster}_피해액'][j] = 0
    
    df['총_인명피해'] = df['호우_인명피해'] + df['대설_인명피해']
    df['총_피해액'] = df['호우_피해액'] + df['대설_피해액']

    return df

피해규모 데이터에 있는 인명피해와 피해액 컬럼을 앞서 정의한 mpc_year 함수로 생성한 df_mpc으로 이동시킵니다.

피해규모 데이터는 피해가 발생한 기간을 보여주는 데이터인 반면에 df_mpc는 일별 데이터이기 때문에 between을 이용하여 범위를 지정합니다.

해당 기간의 인명피해 및 피해액은 해당 자연재난과의 상관계수가 가장 높은 지표의 비율에 따라 나누었습니다.

# 4. 전처리 함수 실행

In [8]:
# 호우
df_mpc = mpc_year(df)
for i in range(len(dmg_data_path)):
    df_damage = nd_dmg(dmg_data_path[i], df_mpc, '호우', '일강수량')
    
# 대설
for i in range(len(dmg_data_path)):
    df_dmg_final = nd_dmg(dmg_data_path[i], df_damage, '대설', '일_최심신적설')

In [9]:
df_dmg_final['호우_피해액'] = df_dmg_final['호우_피해액'] * 1000
df_dmg_final['대설_피해액'] = df_dmg_final['대설_피해액'] * 1000
df_dmg_final['총_피해액'] = df_dmg_final['총_피해액'] * 1000

피해액 컬럼의 단위를 수정합니다.

In [10]:
df_dmg_final.to_csv('./data/df_damage.csv', index=False)